## Report: Analyzing Boston AirBnB Data

This project involves creating SQL views on the `bnb.db` database, associated with the problem set available at [https://cs50.harvard.edu/sql/2024/psets/4/bnb/](https://cs50.harvard.edu/sql/2024/psets/4/bnb/), to understand the impact of AirBnB on the local tourism scene in Boston. The `bnb.db` database contains information sourced directly from AirBnB, structured across three tables: `listings`, `reviews`, and `availabilities`. By constructing specific views, we aim to gain insights into various aspects of AirBnB properties, their reviews, and their availability within the city.

## Database Schema: `bnb.db`

The `bnb.db` database comprises three tables designed to capture different facets of AirBnB listings in Boston. The schema for each table is outlined below:

### `listings` Table

| Column Name   | Data Type | Description                                                                 |
|---------------|-----------|-----------------------------------------------------------------------------|
| `id`          | INTEGER   | Unique identifier for each AirBnB listing.                                |
| `property_type`| TEXT      | The type of property being listed (e.g., "Entire rental unit", "Private room"). |
| `host_name`   | TEXT      | The AirBnB username of the host who owns the listing.                      |
| `accommodates`| INTEGER   | The maximum number of guests the listing can accommodate.                  |
| `bedrooms`    | INTEGER   | The number of bedrooms available in the listing.                           |
| `description` | TEXT      | The textual description provided for the listing on AirBnB.                |

### `reviews` Table

| Column Name   | Data Type | Description                                                        |
|---------------|-----------|--------------------------------------------------------------------|
| `id`          | INTEGER   | Unique identifier for each review.                               |
| `listing_id`  | INTEGER   | Foreign key referencing the `id` in the `listings` table.           |
| `date`        | TEXT      | The date when the review was posted.                             |
| `reviewer_name`| TEXT      | The AirBnB username of the user who wrote the review.               |
| `comments`    | TEXT      | The textual content of the review.                               |

### `availabilities` Table

| Column Name | Data Type | Description                                                               |
|-------------|-----------|---------------------------------------------------------------------------|
| `id`        | INTEGER   | Unique identifier for each availability record.                         |
| `listing_id`| INTEGER   | Foreign key referencing the `id` in the `listings` table.            |
| `date`      | TEXT      | The date for which availability is being recorded.                       |
| `available` | BOOLEAN   | Indicates if the listing is available for booking on the given date (TRUE/FALSE). |
| `price`     | REAL      | The price of staying at the listing on the given date.                   |

## Project Specification: Creating SQL Views

The project requires the creation of several SQL views based on the data within these tables. Each view should be designed to provide a specific perspective on the AirBnB data in Boston. It's important to note that each view must be self-contained and not rely on the existence of any other previously created view.

In [16]:
import sqlite3
import pandas as pd

### Connecting to the Database

In [17]:
connection = sqlite3.connect("data_bases/bnb.db")
print("Connected to the database!")

Connected to the database!


Dropping Views if they already exist

In [18]:
cursor = connection.cursor()

drop_views = """
DROP VIEW IF EXISTS frequently_reviewed;
DROP VIEW IF EXISTS available;
DROP VIEW IF EXISTS june_vacancies;
DROP VIEW IF EXISTS no_descriptions;
DROP VIEW IF EXISTS one_bedrooms;
"""

cursor.executescript(drop_views)

# Commit and close
connection.commit()

1. Write a SQL statement to create a view named no_descriptions that includes all of the columns in the listings table except for description.

In [19]:
cursor.execute("""
CREATE VIEW IF NOT EXISTS no_descriptions AS
SELECT id, property_type, host_name, accommodates, bedrooms
FROM listings;
""")

connection.commit()

In [20]:
query = """
SELECT *
FROM "no_descriptions"
LIMIT 5
;
"""

df = pd.read_sql_query(query, connection)

df

,id,property_type,host_name,accommodates,bedrooms
0,3781,Entire rental unit,Frank,2,1.0
1,5506,Entire guest suite,Terry,2,1.0
2,6695,Entire condo,Terry,4,NaN
3,8789,Entire rental unit,Anne,2,1.0
4,10813,Entire rental unit,Michelle,2,NaN


2. Write a SQL statement to create a view named one_bedrooms. This view should contain all listings that have exactly one bedroom. Ensure the view contains the following columns:

* id, which is the id of the listing from the listings table.
* property_type, from the listings table.
* host_name, from the listings table.
* accommodates, from the listings table.

In [21]:
cursor.execute("""
CREATE VIEW IF NOT EXISTS "one_bedrooms" AS
SELECT "id", "property_type", "host_name" "accommodates"
FROM "listings"
WHERE "bedrooms" = 1;
""")

connection.commit()

In [22]:
query = """
SELECT *
FROM "one_bedrooms"
LIMIT 5
;
"""

df = pd.read_sql_query(query, connection)

df

,id,property_type,accommodates
0,3781,Entire rental unit,Frank
1,5506,Entire guest suite,Terry
2,8789,Entire rental unit,Anne
3,29765,Entire rental unit,Elizabeth
4,45987,Entire rental unit,Atef


3. Write a SQL statement to create a view named available. This view should contain all dates that are available at all listings. Ensure the view contains the following columns:

* id, which is the id of the listing from the listings table.
* property_type, from the listings table.
* host_name, from the listings table.
* date, from the availabilities table, which is the date of the availability.

In [23]:
cursor.execute("""
CREATE VIEW IF NOT EXISTS "available" AS
SELECT "listings"."id", "listings"."property_type", "listings"."host_name","availabilities"."date"
FROM "listings"
JOIN "availabilities" ON "listings"."id" = "availabilities"."listing_id"
WHERE "availabilities"."available" = 'TRUE';
""")

connection.commit()

In [24]:
query = """
SELECT *
FROM "available"
LIMIT 5
;
"""

df = pd.read_sql_query(query, connection)

df

,id,property_type,host_name,date
0,3781,Entire rental unit,Frank,2023-07-14
1,3781,Entire rental unit,Frank,2023-07-15
2,3781,Entire rental unit,Frank,2023-07-16
3,3781,Entire rental unit,Frank,2023-07-17
4,3781,Entire rental unit,Frank,2023-07-18


4. write a SQL statement to create a view named june_vacancies. This view should contain all listings and the number of days in June of 2023 that they remained vacant. Ensure the view contains the following columns:

* id, which is the id of the listing from the listings table.
* property_type, from the listings table.
* host_name, from the listings table.
* days_vacant, which is the number of days in June of 2023, that the given listing was marked as available.

In [25]:
cursor.execute("""
CREATE VIEW IF NOT EXISTS "june_vacancies" AS
SELECT "listings"."id", "listings"."property_type", "listings"."host_name",
COUNT("availabilities"."date") AS "days_vacant"
FROM "listings"
JOIN "availabilities" ON "listings"."id" = "availabilities"."listing_id"
WHERE "availabilities"."available" = 'TRUE'
AND "availabilities"."date" >= '2023-06-01'
AND "availabilities"."date" <= '2023-06-31'
GROUP BY "listings"."id";
""")

connection.commit()

In [26]:
query = """
SELECT *
FROM "june_vacancies"
LIMIT 5
;
"""

df = pd.read_sql_query(query, connection)

df

,id,property_type,host_name,days_vacant
0,10813,Entire rental unit,Michelle,9
1,10986,Entire condo,Michelle,9
2,67774,Entire condo,Anne,1
3,184893,Private room in bed and breakfast,Dawn,9
4,210097,Entire home,Maria Elena,2


5. write a SQL statement to create a view named frequently_reviewed. This view should contain the 100 most frequently reviewed listings, sorted from most- to least-frequently reviewed. Ensure the view contains the following columns:

* id, which is the id of the listing from the listings table.
* property_type, from the listings table.
* host_name, from the listings table.
* reviews, which is the number of reviews the listing has received.

If any two listings have the same number of reviews, sort by property_type (in alphabetical order), followed by host_name (in alphabetical order).

In [27]:
cursor.execute("""
CREATE VIEW IF NOT EXISTS "frequently_reviewed" AS
SELECT "listings"."id", "listings"."property_type", "listings"."host_name",
COUNT("reviews"."id") AS "reviews"
FROM "listings"
JOIN "reviews" ON "listings"."id" = "reviews"."listing_id"
GROUP BY "listings"."id"
ORDER BY "reviews" DESC
LIMIT 100;
""")

connection.commit()

In [28]:
query = """
SELECT *
FROM "frequently_reviewed"
LIMIT 5
;
"""

df = pd.read_sql_query(query, connection)

df

,id,property_type,host_name,reviews
0,4090224,Entire home,Tiffany,860
1,18290558,Private room in home,Boris & Susan,786
2,815639,Entire condo,Jason,773
3,916123,Entire condo,Jason,767
4,18584891,Private room in home,Roger Michael &Quot;Corey&Quot;,733


In [29]:
connection.close()